# House Pricing

## Problem Statement

We would like to predict the house of a house based on its characteristics.

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("house_prices.csv")

In [3]:
df.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  
3   2006        WD        Abnorml     140000  
4   2008        WD         Normal     250000  

[5 rows x 81 columns]

In [4]:
df.columns[df.dtypes==int64]

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [5]:
X = df[['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']]
y = df['SalePrice']

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [133]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
X_val,X_test,y_val,y_test = train_test_split(X_test,y_test, test_size=0.5)

X_t = X_train.append(X_test)
y_t = y_train.append(y_test)

In [134]:
best_score = pd.DataFrame(index=['MSE'])

In [135]:
from sklearn.linear_model import LinearRegression

In [136]:
lr = LinearRegression()
error = np.abs(cross_val_score(lr, X_t,y_t, cv=5, scoring='neg_mean_squared_error').mean())
error

1528675139.4273055

In [137]:
best_score['LinearReg']= [error]
best_score

LinearReg
MSE  1.528675e+09

In [138]:
from sklearn.tree import DecisionTreeRegressor

In [139]:
tree = RandomizedSearchCV(DecisionTreeRegressor(),param_distributions={'max_leaf_nodes':np.arange(10,100,10),
                                                                       'max_depth':np.arange(1,20)}, 
                                                                      cv= 5, scoring='neg_mean_squared_error')

In [140]:
tree.fit(X_t,y_t)

/home/carcrupe/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=DecisionTreeRegressor(criterion='mse',
                                                   max_depth=None,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort=False,
                                                   random_state=None,
                                                   splitter='best'),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_di

In [141]:
bTree = tree.best_estimator_.fit(X_train,y_train)
print(np.abs(tree.best_score_))

1449035362.4850595


In [142]:
tree.best_score_

-1449035362.4850595

In [143]:
error = np.abs(cross_val_score(bTree, X_t,y_t, cv=5, scoring='neg_mean_squared_error').mean())
error

1536429909.25222

In [144]:
best_score['DecTree']= [error]
best_score

LinearReg       DecTree
MSE  1.528675e+09  1.536430e+09

In [145]:
from sklearn.ensemble import RandomForestRegressor

In [146]:
rf = RandomizedSearchCV(RandomForestRegressor(),param_distributions={'max_leaf_nodes':np.arange(10,100,10),
                                                                       'max_depth':np.arange(1,20), 'n_estimators':np.arange(10,200,10)}, 
                                                                      cv= 5, scoring='neg_mean_squared_error')

In [147]:
rf.fit(X_t,y_t)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [148]:
bRf = rf.best_estimator_.fit(X_train,y_train)
print(np.abs(rf.best_score_))

966506297.531715


In [149]:

error = np.abs(cross_val_score(bRf, X_t,y_t, cv=5, scoring='neg_mean_squared_error').mean())
error

1000272133.600321

In [150]:
best_score['RanForest']= [error]
best_score

LinearReg       DecTree     RanForest
MSE  1.528675e+09  1.536430e+09  1.000272e+09

In [159]:
from sklearn.ensemble import GradientBoostingRegressor


In [160]:
gbr = GradientBoostingRegressor(max_depth=4, n_estimators=100,learning_rate=0.1)

In [161]:
bgbr = RandomizedSearchCV(gbr, param_distributions={'max_depth':np.arange(1,20),'n_estimators':np.arange(10,100,10), 
                                                   'learning_rate':np.arange(0.1,1,0.1)}, cv=3, n_iter=20)

In [162]:
bgbr.fit(X_t,y_t)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=4,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                            

In [163]:
be = bgbr.best_estimator_.fit(X_train,y_train)

In [166]:
error = np.abs(cross_val_score(be, X_t,y_t, cv=5, scoring='neg_mean_squared_error').mean())
error

875720169.4505155

In [167]:
best_score['GradBoosting']= [error]
best_score

LinearReg       DecTree     RanForest  GradBoosting
MSE  1.528675e+09  1.536430e+09  1.000272e+09  8.757202e+08

In [168]:
### Best model to predict is GradientBoosting, let us predict with the validation set

In [170]:
best_score_sq = np.sqrt(best_score)
best_score_sq.index = ['RMSE']
best_score_sq

LinearReg       DecTree     RanForest  GradBoosting
RMSE  39098.275402  39197.320179  31627.079119  29592.569497

In [175]:
pred = be.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val,pred))
mse = mean_squared_error(y_val,pred)
print('Mean squared error: ' , mse)
print('Root mean squared error: ' , rmse)
print('Score: ', be.score(X_val,y_val))

Mean squared error:  416259184.2636755
Root mean squared error:  20402.430842026533
Score:  0.9385866074783501


In [176]:
pred = bRf.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val,pred))
mse = mean_squared_error(y_val,pred)
print('Mean squared error: ' , mse)
print('Root mean squared error: ' , rmse)
print('Score: ', bRf.score(X_val,y_val))

Mean squared error:  515978479.75626457
Root mean squared error:  22715.159690309567
Score:  0.9238743789736464
